In [ ]:
import pandas as pd
import os
import pickle
import numpy as np
import json
import pandas as pd

from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt
import itertools


In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion matrix',cmap=plt.cm.Blues, experiment=None, out_path=None):
    #This function prints and plots the confusion matrix. Normalization can be applied by setting `normalize=True`.
    fig_cm = plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap, aspect='auto')
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.3f' 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.tight_layout()    

    if experiment:
        experiment.upload(fig_cm)
    if out_path:
        plt.savefig(out_path)
    plt.show()
    plt.close(fig_cm)

In [ ]:
mount_point = "/mnt/raid/C1_ML_Analysis/"

In [ ]:
model_output_dir = "test_output/classification/effnetv2s/ac/v0.12/epoch=42-val_loss=0.34"

df_pred = pd.read_csv(os.path.join(model_output_dir, "c3_blindsweep_annotation_labels_test_prediction.csv"))
pred = pickle.load(open(os.path.join(model_output_dir, "c3_blindsweep_annotation_labels_test_predictions.pickle"), 'rb'))
probs = pickle.load(open(os.path.join(model_output_dir, "c3_blindsweep_annotation_labels_test_probs.pickle"), 'rb'))


In [ ]:
df_annot = pd.read_csv(os.path.join(mount_point, "CSV_files/c3_blindsweep_annotation_labels_merged.csv"))

In [ ]:
df_pred.iloc[0]

In [ ]:
df_pred.columns

In [ ]:

id_column = 'annotation_id'
frame_column = 'frame_index'
frame_label = 'annotation_label'

frame_labels_dict = {
    'low_visible': 0,
    'high_visible': 0,
    'low_measurable': 0,
    'high_measurable': 1
}

targets = []
predictions = []


for idx, row in df_pred.iterrows():

    n_frames = pred[idx].shape[0]
    uid = row[id_column]

    frames = df_annot[df_annot[id_column] == uid].sort_values(by=frame_column).reset_index(drop=True)

    frame_idx = frames[frame_column].to_numpy()
    frame_idx = np.clip(frame_idx, 0, n_frames - 1)

    frame_labels = frames[frame_label].values.tolist()
    frame_labels_idx = np.array([frame_labels_dict[lbl] for lbl in frame_labels])

    img_labels = np.zeros(n_frames)
    img_labels[frame_idx] = frame_labels_idx


    targets.extend(img_labels.tolist())
    predictions.extend(pred[idx].tolist())


print(classification_report(targets, predictions, digits=3))

In [ ]:
cnf_matrix = confusion_matrix(targets, predictions, normalize='true')
plot_confusion_matrix(cnf_matrix, classes=['rest', 'ac'], title='Confusion matrix')